This analysis confirms a bug that showed up in private testnet testing. One of the branches of `getSupportPrice()` has a bug that bricks the market if triggered.

In [1]:
WEI = 1
GWEI = 10**9
WHOLE = 10**18

def wei_to_whole(amount):
    return amount // 10**18

def gwei_to_whole(amount):
    return amount // 10**9

def gwei_to_wei(amount):
    return amount * 10**9

def wei_to_gwei(amount):
    return amount // 10**9

def convert_support_price(price):
    """Support price is ETH wei / MKT gwei. Convert to ETH whole / MKT whole"""
    if price >= 10**9:
        # ETH gwei / MKT gwei
        price_gwei_to_gwei = wei_to_gwei(price)
        # Note units cancel
        price_whole_to_whole = price_gwei_to_gwei
        return price_whole_to_whole
    else:
        # price is less than 1 gwei ETH to purchase 1 gwei MKT
        num_gwei_purchased = WHOLE // price
        num_mkt_purchased = num_gwei_purchased // 10**9
        return 1.0 / num_mkt_purchased

In [2]:
PRICE_FLOOR = int(.001 * GWEI)
SPREAD = 110 # 10 %
LIST_REWARD = .000025 * WHOLE # MKT
STAKE = .01 * WHOLE # MKT
VOTE_BY = 7 # Days. Are units right?
BACKEND_PAYMENT = 5
MAKER_PAYMENT = 25
RESERVE_PAYMENT = 100 - BACKEND_PAYMENT - MAKER_PAYMENT
COST_PER_BYTE = 100 * GWEI # ETH

Here we have the creator block set to 0.

In [3]:
# Initial creator transfer into reserve
CREATOR_SUPPORT = 0 * WHOLE # ETH
# Creator does their thing
CREATOR_BLOCK_SIZE = int(1 * WHOLE) # MKT

In [4]:
N_PATRONS = 10
# Support each patron provides
SUPPORT = 1000 * WHOLE # ETH
N_MAKERS = 1000
# This is number of listings *per* maker
N_LISTINGS_PER_MAKER = 100
# Size of an individual data purchase in ETH
PURCHASE_SIZE = 1000 * WHOLE # ETH
# Number of purchases
N_PURCHASES = 90

In [5]:
RESERVE = 0 # Initial reserve is empty
MARKET_TOTAL = 0 # No Market tokens at start

MARKET_TOTAL += CREATOR_BLOCK_SIZE
# Creator deposits funds in reserve
RESERVE += CREATOR_SUPPORT
print("MARKET_TOTAL: %d, RESERVE: %d" % (wei_to_whole(MARKET_TOTAL), wei_to_whole(RESERVE)))

MARKET_TOTAL: 1, RESERVE: 0


In [6]:
coin_table = {}
# We're just going to use whole units for convenience
coin_table["CREATOR"] = CREATOR_BLOCK_SIZE / 10**18

In [7]:
print("SUPPORT: %d" % wei_to_whole(SUPPORT))
for i in range(N_PATRONS):
    print("Patron %d supports" % (i+1))
    if MARKET_TOTAL < 1 * WHOLE:
        SUPPORT_PRICE = PRICE_FLOOR + ((SPREAD * RESERVE * GWEI) // (100 * 1 * WHOLE))
    else:
        SUPPORT_PRICE = PRICE_FLOOR + ((SPREAD * RESERVE * GWEI) // (100 * MARKET_TOTAL))
    print("PRICE_FLOOR: %d" % PRICE_FLOOR)
    print("RAW SUPPORT_PRICE: %d" % SUPPORT_PRICE)
    print("SUPPORT_PRICE (ETH whole for MKT whole): %f" % convert_support_price(SUPPORT_PRICE))
    MINTED = (SUPPORT // SUPPORT_PRICE) * GWEI # Units of WEI
    print("MINTED: %f" % (MINTED / 10**18))
    MARKET_TOTAL += MINTED
    RESERVE += SUPPORT
    coin_table["PATRON_%d" % (i+1)] = MINTED / 10**18

print("############################")
print("MARKET_TOTAL: %f" % (MARKET_TOTAL / 10**18))
print("RESERVE: %f" % (RESERVE / 10**18))
print("coin_table")
print(coin_table)

SUPPORT: 1000
Patron 1 supports
PRICE_FLOOR: 1000000
RAW SUPPORT_PRICE: 1000000
SUPPORT_PRICE (ETH whole for MKT whole): 0.001000
MINTED: 1000000.000000
Patron 2 supports
PRICE_FLOOR: 1000000
RAW SUPPORT_PRICE: 2099998
SUPPORT_PRICE (ETH whole for MKT whole): 0.002101
MINTED: 476190.929706
Patron 3 supports
PRICE_FLOOR: 1000000
RAW SUPPORT_PRICE: 2490321
SUPPORT_PRICE (ETH whole for MKT whole): 0.002494
MINTED: 401554.659018
Patron 4 supports
PRICE_FLOOR: 1000000
RAW SUPPORT_PRICE: 2757425
SUPPORT_PRICE (ETH whole for MKT whole): 0.002762
MINTED: 362657.189225
Patron 5 supports
PRICE_FLOOR: 1000000
RAW SUPPORT_PRICE: 2963931
SUPPORT_PRICE (ETH whole for MKT whole): 0.002967
MINTED: 337389.770545
Patron 6 supports
PRICE_FLOOR: 1000000
RAW SUPPORT_PRICE: 3133607
SUPPORT_PRICE (ETH whole for MKT whole): 0.003135
MINTED: 319121.064001
Patron 7 supports
PRICE_FLOOR: 1000000
RAW SUPPORT_PRICE: 3278285
SUPPORT_PRICE (ETH whole for MKT whole): 0.003279
MINTED: 305037.542496
Patron 8 supports
P